# Funciones de orden superior

In [2]:
// Importamos librerias para testing
import $ivy.`org.scalatest::scalatest:3.0.8`
import _root_.org.scalatest._

import $ivy.$                               

import _root_.org.scalatest._

## `FoldRight`: estrategia para algoritmos de divide y vencerás

Consideremos dos funciones que aplican un patrón de divide y vencerás

In [3]:
def sum(list: List[Int]): Int = 
    list match {
        case Nil => 0
        case head :: tail => head + sum(tail)
    }

defined function sum

In [4]:
def multiply(list: List[Int]): Int = 
    list match {
        case Nil => 1
        case head :: tail => head * multiply(tail)
    }

defined function multiply

Ambas funciones son éxactamente iguales en cuanto a su estructura y en lo único que varian es en el elemento que es devuelto en caso de `Nil` y el operador que usamos en cada caso (`+` o `*`), así pues podemos abstraer esta estrcutura en un método mucho más general:

In [5]:
def combine(list: List[Int])(nil: Int, cons: (Int, Int) => Int): Int = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, combine(tail)(nil, cons))
    }

defined function combine

Este método abstrae la estrucutra principal de nuestras funciones, de modo que sus argumentos son la estructura sobre la que operamos (una lista en nuestro caso), el elemento a devolver en caso de `Nil` y una función lambda que describa el comportamiento específico

Así podemos redefinir el comportamiento de las funciones `sum` y `multiply`

In [6]:
// Podemos definir la función lambda de forma normal
def sum(list: List[Int]): Int = 
    combine(list)(0, (a, b) => a + b)

defined function sum

In [7]:
// O podemos definirla abreviada
def multiply(list: List[Int]): Int = 
    combine(list)(1, _ * _)

defined function multiply

Podemos así generalizar nuestra función `combine` y obtener así la función `foldRight`

In [8]:
def foldRight[A, B](list: List[A])(nil: B, cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil, cons))
    }

defined function foldRight

### Funcionamiento de `foldRight`

Gráficamente el comportamiento de `foldRight` se muestra como sigue:

Podemos así entender el funcionamiento de `foldRight` como una implementación del algoritmo de divide y vencerás, es decir, primero se divide el problema en subproblemas más pequeños y faciles de resolver, de modo que en segundo lugar, estos problemas son resueltos y finalmente, todos los problemas son combinados para obtener la solución final. En el caso de que los subproblemas no puedan ser resultos de forma directa, se resolverán de forma recursiva. En el caso de las listas podemos describir el siguiente comportamiento:\
- El problema es obtener un valor de tipo $B$ de una lista dada.
- El único subproblema le corresponde a la cola de la lista.
- Los argumentos de `foldRight` nos indican por tanto cómo obtener la solucion para una lista vacia (problema atómico), y cómo obtener la solución para el subproblema 

Así de este modo, podemos implementar las funciones `sum` y `multiply` mediante `foldRight`

In [9]:
def sum(list: List[Int]): Int = 
    foldRight[Int, Int](list)(
        0, // Solución al problema atómico
        (head, subsol) => head + subsol // composición para la resolución del subproblema
    )

defined function sum

In [10]:
def multiply(list: List[Int]): Int = 
    foldRight(list)(
        1, // Solución al problema atómico
        (head: Int, subsol: Int) => head * subsol // composición para la resolución del subproblema
    )

defined function multiply

### Mejora en la inferencia de tipos

Como podemo apreciar, necesitamos definir los tipos de nuestra función lambda para que Scala pueda compilar. Así pues una definición de este tipo no compilaria.

In [10]:
/*
def multiply(list: List[Int]): Int = 
    foldRight(list)(1, (a, b) => a * b)
*/

Para poder ayudar a Scala a inferir los tipos, podemos cambiar la signatura de `foldRight` currificandola por completo

In [11]:
def foldRight[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil)(cons))
    }

defined function foldRight

Así de esta manera, la definición anterior consigue compilar

In [13]:
def multiply(list: List[Int]): Int = 
    foldRight(list)(1)((a, b) => a * b)

defined function multiply

### Funciones de orden superior en la API de Scala